<a href="https://colab.research.google.com/github/rianachatterjee04/GenAssist/blob/main/Imagine_Segmentation_and_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies

In [ ]:
!pip install ultralytics

!pip install yt-dlp

!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Downloading Youtube Video

In [ ]:
import os

os.makedirs("videos", exist_ok = True)

VIDEO_URL = "https://www.youtube.com/watch?v=RFb6wbsffY0"
!yt-dlp -o "videos/%(new_york)s.%(ext)s" {VIDEO_URL}

!ls videos

[youtube] Extracting URL: https://www.youtube.com/watch?v=RFb6wbsffY0
[youtube] RFb6wbsffY0: Downloading webpage
[youtube] RFb6wbsffY0: Downloading tv client config
[youtube] RFb6wbsffY0: Downloading player 0f7c1eff
[youtube] RFb6wbsffY0: Downloading tv player API JSON
[youtube] RFb6wbsffY0: Downloading ios player API JSON
[youtube] RFb6wbsffY0: Downloading m3u8 information
[info] RFb6wbsffY0: Downloading 1 format(s): 315+251
[download] Destination: videos/NA.f315.webm
[download] 100% of    1.07GiB in 00:00:32 at 34.17MiB/s
[download] Destination: videos/NA.f251.webm
[download] 100% of    4.66MiB in 00:00:00 at 32.67MiB/s
[Merger] Merging formats into "videos/NA.webm"
Deleting original file videos/NA.f315.webm (pass -k to keep)
Deleting original file videos/NA.f251.webm (pass -k to keep)
NA.webm


In [ ]:
import os

video_path = "videos/NA.webm"

# Get absolute path
abs_path = os.path.abspath(video_path)
print(f"Checking video path: {abs_path}")

# Verify if file exists
if os.path.exists(video_path):
    print("✅ Video file exists!")
else:
    print("❌ Video file NOT found. Check the file name and location.")

Checking video path: /content/videos/NA.webm
✅ Video file exists!


Extracting Relevant Frames

In [ ]:
import cv2
import os

# Path to video file
video_path = "videos/NA.webm"  # Replace with your actual video filename
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Open the video file
vidcap = cv2.VideoCapture(video_path)

# Check if the video was opened successfully
if not vidcap.isOpened():
    print("❌ Error: Could not open video. Check file path and format.")
    exit()

# Get video properties
fps = int(vidcap.get(cv2.CAP_PROP_FPS))  # Frames per second
total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frame count
duration = total_frames / fps  # Video duration in seconds

print(f"Video FPS: {fps}")
print(f"Total Frames: {total_frames}")
print(f"Duration: {duration:.2f} seconds")

# Set extraction range (in seconds)
start_sec, end_sec = 7, 67  # Extract frames from 7s to 67s
start_frame = start_sec * fps
end_frame = min(end_sec * fps, total_frames)  # Ensure it doesn't exceed video length

# Set video position to the start frame
vidcap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

frame_rate = 30  # Extract one frame every 30 frames
count = 0  # Number of extracted frames

# Loop through frames
while vidcap.isOpened():
    frame_id = int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))  # Current frame index

    # Stop if we reach the end of the desired segment
    if frame_id > end_frame:
        break

    success, image = vidcap.read()
    if not success:
        print("❌ Error: Could not read frame.")
        break  # Stop if reading fails

    # Extract every `frame_rate` frames
    if frame_id % frame_rate == 0:
        frame_name = os.path.join(output_dir, f"frame_{frame_id}.jpg")
        cv2.imwrite(frame_name, image)  # Save frame as JPEG
        print(f"✅ Saved: {frame_name}")
        count += 1

vidcap.release()  # Release video capture
print(f"✅ Extracted {count} frames from {start_sec}s to {end_sec}s in '{output_dir}'")


Video FPS: 60
Total Frames: 20344
Duration: 339.07 seconds
✅ Saved: frames/frame_420.jpg
✅ Saved: frames/frame_450.jpg
✅ Saved: frames/frame_480.jpg
✅ Saved: frames/frame_510.jpg
✅ Saved: frames/frame_540.jpg
✅ Saved: frames/frame_570.jpg
✅ Saved: frames/frame_600.jpg
✅ Saved: frames/frame_630.jpg
✅ Saved: frames/frame_660.jpg
✅ Saved: frames/frame_690.jpg
✅ Saved: frames/frame_720.jpg
✅ Saved: frames/frame_750.jpg
✅ Saved: frames/frame_780.jpg
✅ Saved: frames/frame_810.jpg
✅ Saved: frames/frame_840.jpg
✅ Saved: frames/frame_870.jpg
✅ Saved: frames/frame_900.jpg
✅ Saved: frames/frame_930.jpg
✅ Saved: frames/frame_960.jpg
✅ Saved: frames/frame_990.jpg
✅ Saved: frames/frame_1020.jpg
✅ Saved: frames/frame_1050.jpg
✅ Saved: frames/frame_1080.jpg
✅ Saved: frames/frame_1110.jpg
✅ Saved: frames/frame_1140.jpg
✅ Saved: frames/frame_1170.jpg
✅ Saved: frames/frame_1200.jpg
✅ Saved: frames/frame_1230.jpg
✅ Saved: frames/frame_1260.jpg
✅ Saved: frames/frame_1290.jpg
✅ Saved: frames/frame_1320.jpg


Instance Segmentation and Tracking

In [ ]:
import cv2
import os
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

# Install Ultralytics YOLO (if not installed)
!pip install ultralytics --quiet

# Load YOLO model for instance segmentation & tracking
model = YOLO("yolov8n-seg.pt")  # Change to "best.pt" if using a trained model

# Input and output directories
input_frames_dir = "frames"  # Directory where extracted frames are stored
output_frames_dir = "segmented_frames"  # Directory for saving segmented frames
os.makedirs(output_frames_dir, exist_ok=True)

# List all extracted frames
frame_files = sorted(os.listdir(input_frames_dir))

# Process frames for instance segmentation & tracking
for frame_file in frame_files:
    frame_path = os.path.join(input_frames_dir, frame_file)
    frame = cv2.imread(frame_path)

    if frame is None:
        print(f"❌ Error loading frame: {frame_file}")
        continue

    # Run instance segmentation & tracking
    results = model.track(frame, persist=True)

    # Annotator for visualization
    annotator = Annotator(frame, line_width=2)

    if results[0].boxes.id is not None and results[0].masks is not None:
        masks = results[0].masks.xy
        track_ids = results[0].boxes.id.int().cpu().tolist()
        classes = results[0].boxes.cls.cpu().tolist()

        # Draw segmentation masks with object tracking
        for mask, track_id, cls in zip(masks, track_ids, classes):
            color = colors(track_id, True)  # Unique color per tracked object
            annotator.seg_bbox(mask=mask, mask_color=color, label=f"ID {track_id} - {model.model.names[int(cls)]}")

    # Save the segmented frame
    output_path = os.path.join(output_frames_dir, frame_file)
    cv2.imwrite(output_path, frame)
    print(f"✅ Processed & saved: {output_path}")

print(f"✅ All segmented frames saved in '{output_frames_dir}'")

# Zip the segmented frames for easy download
!zip -r segmented_frames.zip segmented_frames/


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.74M/6.74M [00:00<00:00, 83.5MB/s]


Streaming output truncated to the last 5000 lines.
0: 384x640 12 persons, 1 car, 1 bus, 318.6ms
Speed: 5.6ms preprocess, 318.6ms inference, 120.1ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed & saved: segmented_frames/frame_438.jpg

0: 384x640 13 persons, 1 car, 1 bus, 325.0ms
Speed: 4.4ms preprocess, 325.0ms inference, 118.0ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed & saved: segmented_frames/frame_439.jpg

0: 384x640 13 persons, 1 car, 1 bus, 353.9ms
Speed: 9.6ms preprocess, 353.9ms inference, 107.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed & saved: segmented_frames/frame_440.jpg

0: 384x640 13 persons, 1 car, 1 bus, 253.8ms
Speed: 5.9ms preprocess, 253.8ms inference, 51.7ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed & saved: segmented_frames/frame_441.jpg

0: 384x640 12 persons, 1 car, 1 bus, 198.6ms
Speed: 5.3ms preprocess, 198.6ms inference, 41.6ms postprocess per image at shape (1, 3, 384, 640)
✅ Processed & 

Converting the frames into a video

In [ ]:
import cv2
import os
import glob

# Directory containing processed frames
frames_dir = "segmented_frames"
output_video_path = "output_segmented.mp4"

# Get all frames sorted by filename
frame_files = sorted(glob.glob(os.path.join(frames_dir, "*.jpg")))

# Read first frame to get video properties
first_frame = cv2.imread(frame_files[0])
height, width, _ = first_frame.shape
fps = 30  # Set FPS (adjust based on original video)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Write frames into video
for frame_file in frame_files:
    frame = cv2.imread(frame_file)
    video_writer.write(frame)

video_writer.release()
print(f"✅ Segmented video saved as: {output_video_path}")

# Download the final video
from google.colab import files
files.download(output_video_path)


✅ Segmented video saved as: output_segmented.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r segmented_frames.zip segmented_frames/
from google.colab import files
files.download("segmented_frames.zip")


updating: segmented_frames/ (stored 0%)
updating: segmented_frames/frame_1788.jpg (deflated 1%)
updating: segmented_frames/frame_1601.jpg (deflated 1%)
updating: segmented_frames/frame_581.jpg (deflated 1%)
updating: segmented_frames/frame_773.jpg (deflated 0%)
updating: segmented_frames/frame_753.jpg (deflated 1%)
updating: segmented_frames/frame_2576.jpg (deflated 1%)
updating: segmented_frames/frame_1392.jpg (deflated 1%)
updating: segmented_frames/frame_769.jpg (deflated 0%)
updating: segmented_frames/frame_1348.jpg (deflated 1%)
updating: segmented_frames/frame_2529.jpg (deflated 0%)
updating: segmented_frames/frame_2180.jpg (deflated 0%)
updating: segmented_frames/frame_1607.jpg (deflated 1%)
updating: segmented_frames/frame_2897.jpg (deflated 1%)
updating: segmented_frames/frame_2942.jpg (deflated 0%)
updating: segmented_frames/frame_2373.jpg (deflated 1%)
updating: segmented_frames/frame_1755.jpg (deflated 1%)
updating: segmented_frames/frame_2332.jpg (deflated 0%)
updating: se

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Creating a testing dataset

In [ ]:
import cv2
import os

# Path to video file
video_path = "videos/NA.webm"  # Replace with your actual video filename
testing_output_dir = "testing_frames"  # New folder for testing images
os.makedirs(testing_output_dir, exist_ok=True)

# Open the video file
vidcap = cv2.VideoCapture(video_path)

# Check if the video was opened successfully
if not vidcap.isOpened():
    print("❌ Error: Could not open video. Check file path and format.")
    exit()

# Get video properties
fps = int(vidcap.get(cv2.CAP_PROP_FPS))  # Frames per second
total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frame count
duration = total_frames / fps  # Video duration in seconds

print(f"🎥 Video FPS: {fps}")
print(f"📸 Total Frames: {total_frames}")
print(f"⏳ Duration: {duration:.2f} seconds")

# Set extraction range (in seconds)
start_sec, end_sec = 68, 120  # Extract frames from 68s to 120s
start_frame = start_sec * fps
end_frame = min(end_sec * fps, total_frames)  # Ensure it doesn't exceed video length

# Set video position to the start frame
vidcap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

frame_rate = 30  # Extract one frame every 30 frames
count = 0  # Number of extracted frames

# Loop through frames
while vidcap.isOpened():
    frame_id = int(vidcap.get(cv2.CAP_PROP_POS_FRAMES))  # Current frame index

    # Stop if we reach the end of the desired segment
    if frame_id > end_frame:
        break

    success, image = vidcap.read()
    if not success:
        print("❌ Error: Could not read frame.")
        break  # Stop if reading fails

    # Extract every `frame_rate` frames
    if frame_id % frame_rate == 0:
        frame_name = os.path.join(testing_output_dir, f"frame_{frame_id}.jpg")
        cv2.imwrite(frame_name, image)  # Save frame as JPEG
        print(f"✅ Saved: {frame_name}")
        count += 1

vidcap.release()  # Release video capture
print(f"✅ Extracted {count} frames from {start_sec}s to {end_sec}s in '{testing_output_dir}'")

# Zip the extracted frames for easy download (if using Google Colab)
!zip -r testing_frames.zip testing_frames/


🎥 Video FPS: 60
📸 Total Frames: 20344
⏳ Duration: 339.07 seconds
✅ Saved: testing_frames/frame_4080.jpg
✅ Saved: testing_frames/frame_4110.jpg
✅ Saved: testing_frames/frame_4140.jpg
✅ Saved: testing_frames/frame_4170.jpg
✅ Saved: testing_frames/frame_4200.jpg
✅ Saved: testing_frames/frame_4230.jpg
✅ Saved: testing_frames/frame_4260.jpg
✅ Saved: testing_frames/frame_4290.jpg
✅ Saved: testing_frames/frame_4320.jpg
✅ Saved: testing_frames/frame_4350.jpg
✅ Saved: testing_frames/frame_4380.jpg
✅ Saved: testing_frames/frame_4410.jpg
✅ Saved: testing_frames/frame_4440.jpg
✅ Saved: testing_frames/frame_4470.jpg
✅ Saved: testing_frames/frame_4500.jpg
✅ Saved: testing_frames/frame_4530.jpg
✅ Saved: testing_frames/frame_4560.jpg
✅ Saved: testing_frames/frame_4590.jpg
✅ Saved: testing_frames/frame_4620.jpg
✅ Saved: testing_frames/frame_4650.jpg
✅ Saved: testing_frames/frame_4680.jpg
✅ Saved: testing_frames/frame_4710.jpg
✅ Saved: testing_frames/frame_4740.jpg
✅ Saved: testing_frames/frame_4770.jpg

Unzip training dataset

In [ ]:
import os
import zipfile

# Unzip the training dataset
train_zip_path = "/content/train.zip"  # Change if your ZIP file is named differently
dataset_path = "/content/dataset"

with zipfile.ZipFile(train_zip_path, "r") as zip_ref:
    zip_ref.extractall(dataset_path)

print(f"✅ Dataset extracted to {dataset_path}")

# Verify the structure
os.system(f"ls {dataset_path}/train")


✅ Dataset extracted to /content/dataset


0

Data labeling

In [ ]:
data_yaml = """
train: /content/dataset/train/images

# Number of classes
nc: 5

# Class names
names: ["obstacles", "person", "road", "sidewalk", "sign"]
"""

# Save the YAML configuration file
with open("data.yaml", "w") as f:
    f.write(data_yaml)

print("✅ data.yaml updated successfully!")


✅ data.yaml updated successfully!


Create validation folders

In [ ]:
import os

# Define paths for validation set
valid_images_dir = "/content/dataset/valid/images"
valid_labels_dir = "/content/dataset/valid/labels"

# Create directories if they don't exist
os.makedirs(valid_images_dir, exist_ok=True)
os.makedirs(valid_labels_dir, exist_ok=True)

print("✅ Created missing validation directories!")


✅ Created missing validation directories!


Moving training to validation data

In [ ]:
import shutil
import random

# Define paths
train_images_dir = "/content/dataset/train/images"
train_labels_dir = "/content/dataset/train/labels"

# Get list of all training images
train_images = os.listdir(train_images_dir)
random.shuffle(train_images)  # Shuffle images to randomize selection

# Select 20% of images for validation
num_valid = int(0.2 * len(train_images))
valid_images = train_images[:num_valid]  # Take first 20% as validation set

# Move images & labels to validation set
for img in valid_images:
    shutil.move(os.path.join(train_images_dir, img), os.path.join(valid_images_dir, img))

    # Move corresponding label (assuming .txt format for YOLO)
    label_file = img.replace(".jpg", ".txt")  # Change extension if needed
    label_path = os.path.join(train_labels_dir, label_file)
    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(valid_labels_dir, label_file))

print(f"✅ Moved {num_valid} images and labels to validation set!")


✅ Moved 10 images and labels to validation set!


In [ ]:
label_file = "/content/dataset/train/labels/" + train_labels[0]  # Open the first label file
with open(label_file, "r") as f:
    label_content = f.readlines()

print(f"📄 First label file content:\n{label_content}")


📄 First label file content:
['3 0.3848958328125 0.9393518515625001 0.3859375 0.9425925921875 0.38619791718749996 0.946759259375 0.38515625 0.9509259265625 0.3786458328125 0.9583333328125001 0.3661458328125 0.9648148140625 0.3609375 0.9703703703125001 0.3583333328125 0.9768518515625001 0.35625 0.991203703125 0.3536458328125 0.9995370375 0.5278645828124999 0.9995370375 0.5252604171875 0.9921296296875 0.521875 0.9861111109375 0.5057291671875 0.9699074078125 0.49296875 0.959722221875 0.48671875 0.95 0.4809895828125 0.9370370375 0.47838541718750005 0.934722221875 0.46666666718750005 0.9300925921875001 0.43151041718750005 0.9254629624999999 0.41614583281249995 0.9254629624999999 0.40026041718750005 0.93101851875 0.390625 0.9361111109375001 0.3828125 0.9370370375 0.3848958328125 0.9393518515625001']


Remove bounding boxes for segmentation

In [ ]:
import os

train_labels_dir = "/content/dataset/train/labels"
valid_labels_dir = "/content/dataset/valid/labels"

# Remove labels that contain only bounding boxes (detection dataset)
for label_file in os.listdir(train_labels_dir):
    with open(os.path.join(train_labels_dir, label_file), "r") as f:
        lines = f.readlines()

    # Check if any line contains only bounding box data (4 coordinates)
    if all(len(line.split()) == 5 for line in lines):  # Format: class_id x_center y_center width height
        os.remove(os.path.join(train_labels_dir, label_file))

for label_file in os.listdir(valid_labels_dir):
    with open(os.path.join(valid_labels_dir, label_file), "r") as f:
        lines = f.readlines()

    if all(len(line.split()) == 5 for line in lines):
        os.remove(os.path.join(valid_labels_dir, label_file))

print("✅ Removed non-segmentation labels.")


✅ Removed non-segmentation labels.


In [ ]:
label_file = "/content/dataset/train/labels/" + os.listdir("/content/dataset/train/labels")[0]
with open(label_file, "r") as f:
    label_content = f.readlines()

print(f"📄 First segmentation label file content:\n{label_content}")


📄 First segmentation label file content:
['3 0.3848958328125 0.9393518515625001 0.3859375 0.9425925921875 0.38619791718749996 0.946759259375 0.38515625 0.9509259265625 0.3786458328125 0.9583333328125001 0.3661458328125 0.9648148140625 0.3609375 0.9703703703125001 0.3583333328125 0.9768518515625001 0.35625 0.991203703125 0.3536458328125 0.9995370375 0.5278645828124999 0.9995370375 0.5252604171875 0.9921296296875 0.521875 0.9861111109375 0.5057291671875 0.9699074078125 0.49296875 0.959722221875 0.48671875 0.95 0.4809895828125 0.9370370375 0.47838541718750005 0.934722221875 0.46666666718750005 0.9300925921875001 0.43151041718750005 0.9254629624999999 0.41614583281249995 0.9254629624999999 0.40026041718750005 0.93101851875 0.390625 0.9361111109375001 0.3828125 0.9370370375 0.3848958328125 0.9393518515625001']


Train the YOLOv8 Model

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 segmentation model
model = YOLO("yolov8n-seg.pt")  # Instance segmentation model

# Train the model
model.train(
    data="/content/dataset/data.yaml",  # Same dataset, but uses segmentation masks
    epochs=50,
    imgsz=640,
    task="segment",  # Segmentation task
    batch=16,
    save=True,
    save_period=5
)

print("✅ Segmentation training started successfully!")



Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /content/dataset/train/labels... 35 images, 7 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<00:00, 706.34it/s]

train: New cache created: /content/dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/valid/labels... 9 images, 1 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 655.02it/s]

val: New cache created: /content/dataset/valid/labels.cache
Plotting labels to runs/segment/train8/labels.jpg... 


...

      50/50         0G     0.6994       1.14      1.951      1.041         11        640: 100%|██████████| 3/3 [00:52<00:00, 17.51s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.58s/it]

                   all         10         10      0.978      0.984      0.995      0.875      0.978      0.984      0.995      0.832



50 epochs completed in 0.850 hours.
Optimizer stripped from runs/segment/train8/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train8/weights/best.pt, 6.8MB

Validating runs/segment/train8/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv8n-seg summary (fused): 195 layers, 3,259,039 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.12s/it]


                   all         10         10      0.896      0.986      0.995      0.879      0.896      0.986      0.995      0.858
                  road          5          6          1      0.971      0.995      0.864          1      0.971      0.995       0.77
              sidewalk          4          4      0.792          1      0.995      0.894      0.792          1      0.995      0.945
Speed: 2.7ms preprocess, 391.8ms inference, 0.0ms loss, 9.2ms postprocess per image
Results saved to runs/segment/train8
✅ Segmentation training started successfully!


Accuracy Metrics and Speed

In [ ]:
import time
import numpy as np
import os
import cv2

# Run validation and measure accuracy
metrics = model.val()
print(metrics)

# Test speed on a batch of images
test_folder = "/content/testing_frames"
test_images = sorted([os.path.join(test_folder, img) for img in os.listdir(test_folder) if img.endswith(".jpg")])

frame_times = []

for img_path in test_images[:100]:  # Test on 100 frames
    img = cv2.imread(img_path)

    start_time = time.time()  # Start time before inference
    results = model.predict(img)  # Run YOLOv8 inference
    end_time = time.time()  # End time after inference

    frame_time = end_time - start_time  # Time per frame
    frame_times.append(frame_time)

# Compute average speed metrics
avg_time_per_frame = np.mean(frame_times)  # Average time per frame (seconds)
fps = 1 / avg_time_per_frame  # Frames per second
total_time = sum(frame_times)  # Total inference time

# Extract segmentation metrics properly
seg_metrics = metrics.seg  # Use segmentation metrics

# Print final performance report
report = f"""
🔥 YOLOv8 Segmentation Performance Report 🔥

🟢 **Segmentation Accuracy Metrics**:
- mAP@50 (segmentation): {seg_metrics.map50:.4f}
- mAP@50-95 (segmentation): {seg_metrics.map:.4f}
- Mean Precision (segmentation): {seg_metrics.mp:.4f}
- Mean Recall (segmentation): {seg_metrics.mr:.4f}

🔵 **Speed Metrics**:
- Total frames processed: {len(frame_times)}
- Total inference time: {total_time:.2f} seconds
- Average time per frame: {avg_time_per_frame:.4f} seconds
- FPS (Frames Per Second): {fps:.2f}

🚀 Model used: yolov8n-seg.pt
"""

print(report)

# Save performance report
with open("performance_report.txt", "w") as f:
    f.write(report)


Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)


val: Scanning /content/dataset/valid/labels.cache... 9 images, 1 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.63s/it]


                   all         10         10      0.896      0.986      0.995      0.879      0.896      0.986      0.995      0.858
                  road          5          6          1      0.971      0.995      0.864          1      0.971      0.995       0.77
              sidewalk          4          4      0.792          1      0.995      0.894      0.792          1      0.995      0.945
Speed: 3.5ms preprocess, 402.1ms inference, 0.0ms loss, 13.4ms postprocess per image
Results saved to runs/segment/train87
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e56f8d40dd0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,   